In [ ]:
# 导入必要的包
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os
# 初始化初始学习率、要训练的epoch数和批大小
INIT_LR = 1e-3
EPOCHS = 25
BS = 32
# 在数据集目录中获取图像列表，然后初始化数据列表(即图像)和类图像
print("[INFO] loading images...")
imagePaths = list(paths.list_images("./sleep_skeleton_dataset"))
data = []
labels = []
# 遍历图像路径
for imagePath in imagePaths:
    # 从文件名中提取类标签
    label = imagePath.split(os.path.sep)[-2]
    # 加载图像，切换颜色通道，并将其调整为固定的224x224像素，同时忽略高宽比
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (260, 260))
    # 分别更新数据和标签列表
    data.append(image)
    labels.append(label)
# 将数据和标签转换为NumPy数组，同时缩放像素强度到范围[0,255]
data = np.array(data) / 255.0
labels = np.array(labels)
# 对标签执行一次热编码
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
# 使用80%的数据用于训练，其余20%用于测试，将数据划分为训练和测试部分
(trainX, testX, trainY, testY) = train_test_split(data, labels,
                                                  test_size=0.20, stratify=labels, random_state=42)
# 初始化训练数据扩充对象
trainAug = ImageDataGenerator(rotation_range=15,
                              fill_mode="nearest")
# 加载VGG16网络，确保关闭head FC层集
baseModel = VGG16(weights="imagenet", include_top=False,
                  input_tensor=Input(shape=(260, 260, 3)))
# 构造模型的头部，它将被放置在基础模型的顶部
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(5, activation="softmax")(headModel)
# 将头部FC模型放在基础模型的顶部(这将成为我们将要训练的实际模型)
model = Model(inputs=baseModel.input, outputs=headModel)
# 循环基础模型中的所有层并冻结它们，这样它们就不会在第一个训练过程中被更新
for layer in baseModel.layers:
    layer.trainable = False
# 编译模型
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
              metrics=["accuracy"])
# 训练网络
print("[INFO] training head...")
H = model.fit_generator(trainAug.flow(trainX, trainY, batch_size=BS),
                        steps_per_epoch=len(trainX) // BS,
                        validation_data=(testX, testY),
                        validation_steps=len(testX) // BS,
                        epochs=EPOCHS)

In [ ]:
# 绘制准确率曲线
plt.plot(H.epoch,H.history.get('accuracy'),label='train_acc')
plt.plot(H.epoch,H.history.get('val_accuracy'),label='test_acc')
plt.legend()
plt.savefig("./训练记录/VGG16/25代_5类_准确率")

In [ ]:
# 保存准确率绘图数据
import scipy.io as scio
data_acc = './训练记录/VGG16/25代_5类_准确率.mat'
scio.savemat(data_acc,{'epoch':H.epoch,'train_acc':H.history.get('accuracy'),'val_acc':H.history.get('val_accuracy')})

In [ ]:
# 绘制误差曲线
plt.plot(H.epoch,H.history.get('loss'),label='train_loss')
plt.plot(H.epoch,H.history.get('val_loss'),label='test_loss')
plt.legend()
plt.savefig("./训练记录/VGG16/25代_5类_误差")

In [ ]:
# 保存误差绘图数据
data_acc = './训练记录/VGG16/25代_5类_误差.mat'
scio.savemat(data_acc,{'epoch':H.epoch,'train_loss':H.history.get('loss'),'val_loss':H.history.get('val_loss')})

In [ ]:
# 保存模型
print("[INFO] saving model...")
model.save("./模型保存/sleep_VGG16_5classify_normal", save_format="h5")